### Desafio: Pessoas em situação de rua

---

**Definição do problema:** Dado que você recebeu dados de pessoas em situação de rua, informe aos pesquisadores as seguintes respostas:  

<ol>
    <li> Em quantas medições, em média, as pessoas da base estava em situação de rua?  </li>
    <li> Qual a proporção de moradores que estiveram inscritos em Section 8 em algum momento? </li> 
    <li> Quantas pessoas estiveram 100% do tempo em situação de rua </li>
    <li> Qual percentual das pessoas apresentaram uma melhora de condições de moradia no final do estudo? </li>
</ol>

**Código do dataset:** housing

**Dicas:**
<ul>
    <li> Cada pessoa está registrada com um ID diferente. </li>
    <li> Uma medição da condição da pessoa é feita a cada seis meses. </li>
    <li> Foram feitas medições por 2 anos. </li>
    <li> É importante conferir a documentação da base para entender o que significa cada código da variável y. </li>
</ul>

---

**Passo 1:** Importar as bibliotecas

In [1]:
%pip install pydataset
import pandas as pd
from pydataset import data 

Note: you may need to restart the kernel to use updated packages.


**Passo 2:** Importar o dataset e verificar sua estrutura 

In [2]:
''' Mostrando a documentação da base de dados. '''
data('housing', show_doc=True)

housing

PyDataset Documentation (adopted from R Documentation. The displayed examples are in R)

##  Homeless Data

### Description

Housing status for 362 severely mentally ill homeless subjects measured at
baseline and at three follow-up times.

### Usage

    data(housing)

### Format

A data frame with 1448 observations on the following 4 variables:

`id`

Subject identifier variable.

`y`

Housing status response, coded as (1) for street living, (2) for community
living and (3) for independent housing.

`time`

Time recorded in months.

`sec`

Section 8 rent certificate indicator.

### Source

Hulrburt M.S., Wood, P.A. and Hough, R.L. (1996). Providing independent
housing for the homeless mentally ill: a novel approach to evaluating
longitudinal housing patterns. _Journal of Community Psychology_, **24**,
291–310.

### Examples

    data(housing)
    str(housing)




In [3]:
''' Leitura da base de dados e imprime as 10 primeiras linhas da tabela ''' 
df = data('housing')
display(df.head(10))

,id,y,time,sec
1,1,1.0,0,1
2,1,2.0,6,1
3,1,2.0,12,1
4,1,2.0,24,1
5,2,1.0,0,1
6,2,2.0,6,1
7,2,2.0,12,1
8,2,1.0,24,1
9,3,0.0,0,1
10,3,2.0,6,1


**Passo 3:** Antes de ir para as perguntas, vamos deixar a base mais amigável

In [9]:
''' Renomeando as colunas usando um dicionario para fazer o de-para. ''' 
df = df.rename(columns={
    'y': 'condicao_moradia',
    'time': 'tempo_medicao',
    'sec': 'inscricao_section_8'
})

df

,id,condicao_moradia,tempo_medicao,inscricao_section_8
1,1,1.0,0,1
2,1,2.0,6,1
3,1,2.0,12,1
4,1,2.0,24,1
5,2,1.0,0,1
...,...,...,...,...
1444,361,NaN,24,0
1445,362,1.0,0,0
1446,362,1.0,6,0
1447,362,1.0,12,0


**Passo 4:** Em quantas medições, em média, as pessoas da base estava em situação de rua? 

In [5]:
# Vamos começar filtrando as observações de pessoas em situação de rua.
df_situacao_rua = df[df['condicao_moradia'] == 1]
# Agrupando a base por medição.
df_situacao_rua = df_situacao_rua.groupby('id')['condicao_moradia'].count()
# Calculando a média de medições em situação de rua.
medicoes_em_rua = df_situacao_rua.mean()
# Imprimindo a saída na tela.
print("Em média, as pessoas da base estava em situação de rua em %.3f medições." 
      % (medicoes_em_rua))

Em média, as pessoas da base estava em situação de rua em 1.891 medições.


**Passo 5:** Qual a proporção de moradores que estiveram inscritos em Section 8 (Assistência Habitacional) em algum momento?

In [6]:
# Agrupando a base por id e somando o valor de inscrição em Section 8.
df_section_8 = df.groupby('id')['inscricao_section_8'].sum()
# Filtrando as pessoas com inscrição e dividindo pelo total, para obter a proporção.
proporcao_section_8 = len(df_section_8[df_section_8 > 0]) / len(df_section_8) * 100
# Imprimindo a saída na tela.
print("A proporção de moradores com ao menos uma inscrição em section 8 é de %.1f%%." 
      % (proporcao_section_8))

A proporção de moradores com ao menos uma inscrição em section 8 é de 50.0%.


**Passo 6:** Quantas pessoas estiveram 100% do tempo em situação de rua?

In [7]:
# Agrupando pelo id e somando a condicao de moradia.
df_situacao_rua_total = df[df['condicao_moradia'] == 1].groupby('id')['condicao_moradia'].count()
# Filtrando os casos em que, em 100% das medições, a pessoa esteve em rua.
situacao_rua_total = len(df_situacao_rua_total[df_situacao_rua_total == 4])
# Imprimindo a saída na tela.
print("%.1d pessoas estiveram em situação de rua em todas as medições." 
      % (situacao_rua_total))

17 pessoas estiveram em situação de rua em todas as medições.


**Passo 7:** Qual percentual das pessoas apresentaram uma melhora de condições de moradia no final do estudo?

In [8]:
# Criando um pivot table para transformar as linhas em colunas.
df_pivot = df.pivot_table(index=['id'], columns='tempo_medicao', values='condicao_moradia')
df_pivot = df_pivot.rename(columns={
    0: 'primeira_medicao',
    6: 'segunda_medicao',
    12: 'terceira_medicao',
    24: 'quarta_medicao',
})
# Filtra as pessoas que apresentaram uma melhroa na condição de moradia entre as medições.
df_pivot_melhoras = df_pivot[df_pivot['quarta_medicao'] > df_pivot['primeira_medicao']]

percentual_melhora = len(df_pivot_melhoras) / len(df_pivot) * 100 
# Imprimindo a saída na tela.
print("%.2f%% pessoas melhoram sua condição de vida entre as medições." 
      % (percentual_melhora))

51.25% pessoas melhoram sua condição de vida entre as medições.
